In [5]:
from patches.patch_utils import get_poisoned_no, save_image, load_image
from patches.agh_patch import get_mask, merge_image_and_mask

from constants import update_dir, PATCHED_DS_PATH
import os
import random
from shutil import copy

In [6]:
poisoned_part = 0.2

In [7]:
patch_name = 'agh_logo_full_visible/' + str(int(poisoned_part * 100))
patch_dir = os.path.join(PATCHED_DS_PATH, patch_name)
os.makedirs(patch_dir, exist_ok=True)

In [8]:
src_class = '1' # if None - all classes are src class
target_class = '39'

In [12]:
trains = {name: [f_name for f_name in os.listdir(update_dir + '/' + name) if os.path.isfile(update_dir + '/' + name + '/' + f_name)] for name in os.listdir(update_dir)}

In [13]:
trains_nos = {key: get_poisoned_no(len(files), poisoned_part) for key, files in trains.items()}

In [15]:
for key, files in trains.items():
    old_class_path = os.path.join(update_dir, key)
    class_path = os.path.join(patch_dir, key)
    target_class_path = os.path.join(patch_dir, target_class)
    
    os.makedirs(class_path, exist_ok=True)
    os.makedirs(target_class_path, exist_ok=True)
    
    to_poison = files[:trains_nos[key]]
    
    # copy all files, also ones that have to be poisoned - we need their pure version too
    for file in files:
        copy(os.path.join(old_class_path, file), os.path.join(class_path, file))
        
    for file in to_poison:
        filename = file[:-4]
        path = os.path.join(old_class_path, file)
        
        if not src_class or src_class == key:
            new_path = os.path.join(target_class_path, filename + '_poisoned.png')
        else:
            new_path = os.path.join(class_path, filename + '_poisoned.png')
        
        file_loaded = load_image(path)
        
        paste_mask, mask = get_mask(file_loaded.size, 0.1, 1, [0.7,0.7], 0)
        poisoned_image = merge_image_and_mask(file_loaded, paste_mask, mask)
        save_image(new_path, poisoned_image)
        
        